In [1]:
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
from tensorflow.keras.callbacks import EarlyStopping

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-09-05 12:59:07.234838: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-05 12:59:07.259235: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[INFO]: sparse_operation_kit is imported


2023-09-05 12:59:08.251676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 12:59:08.251898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 12:59:08.251946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-09-05 12:59:08.479261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 12:59:08.479381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 12:59:08.479429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'train', 'transactions.parquet'),engine="parquet",  part_mem_fraction=0.1)
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'valid', 'transactions.parquet'),engine="parquet", part_mem_fraction=0.1)

In [3]:
customer_features = ['customer_id', 'age', 'FN', 'Active', 'club_member_status','fashion_news_frequency']
article_features = ['article_id', 'product_code', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']

In [4]:
articles = cudf.read_parquet(os.path.join(config.data_raw_dir,'articles.parquet'))[article_features]
customers = cudf.read_parquet(os.path.join(config.data_raw_dir,'customers_enc.parquet'))[customer_features]

In [5]:
articles = nvt.Dataset(articles)
customers = nvt.Dataset(customers)

In [6]:
train = Dataset.merge(train, articles, on="article_id", how="inner")
train = Dataset.merge(train, customers, on="customer_id", how="inner")
# train.regenerate_dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'train.parquet'),part_size="500MiB", file_size="500MiB")
valid = Dataset.merge(valid, articles, on="article_id", how="inner")
valid = Dataset.merge(valid, customers, on="customer_id", how="inner")
# train.regenerate_dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'valid.parquet'),part_size="500MiB", file_size="500MiB")
del articles, customers, train, valid

In [7]:
# ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id','week']

article_id = ["article_id"] >> Categorify() >> TagAsItemID()
customer_id = ["customer_id"] >> Categorify() >> TagAsUserID()


price = ["price"] >> FillMissing(fill_val=0) >> Normalize() >> TagAsItemFeatures() 

time_features = ["t_dat"]
session_time = (
    time_features >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

day = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.day)
        >> nvt.ops.Rename(name = 'day')
        >> Categorify()
        >> TagAsUserFeatures())
week = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.weekday)
        >> nvt.ops.Rename(name = 'week')
        >> Categorify()
        >> TagAsUserFeatures())
month = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.month)
        >> nvt.ops.Rename(name = 'month')
        >> Categorify()
        >> TagAsUserFeatures())
year = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.year)
        >> nvt.ops.Rename(name = 'year')
        >> Categorify()
        >> TagAsUserFeatures())


In [8]:
age_boundaries = list(np.arange(0,100,1))
age =  ['age'] >>  FillMissing(0) >> Bucketize(age_boundaries) >> Categorify() >> TagAsUserFeatures() 
cat_customer_columns = (['FN', 'Active', 'club_member_status','fashion_news_frequency'] 
            >> Categorify() 
            >> TagAsUserFeatures()
            )


In [9]:
# article_features = ['article_id', 'product_code', 'prod_name', 'product_type_no',
#        'product_type_name', 'product_group_name', 'graphical_appearance_no',
#        'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
#        'perceived_colour_value_id', 'perceived_colour_value_name',
#        'perceived_colour_master_id', 'perceived_colour_master_name',
#        'department_no', 'department_name', 'index_code', 'index_name',
#        'index_group_no', 'index_group_name', 'section_no', 'section_name',
#        'garment_group_no', 'garment_group_name', 'detail_desc']

cat_article_columns = (article_features 
              >> Categorify()  
              >> TagAsItemFeatures())

In [10]:
# features = article_id + customer_id + price + day + week + month + year + cat_customer_columns + age + cat_article_columns + time_features
# wf = nvt.Workflow(features)
# train = wf.fit_transform(train)
# valid = wf.transform(valid)
# train.to_parquet(os.path.join(config.data_processed_dir,'train_processed.parquet'))
# valid.to_parquet(os.path.join(config.data_processed_dir,'validation_processed.parquet'))

In [11]:
SESSIONS_MAX_LENGTH =20
SESSIONS_MIN_LENGTH = 5
features = article_id + customer_id + price + day + week + month + year + cat_customer_columns + age + cat_article_columns + time_features
groupby_features = features >> Groupby(
    groupby_cols=["customer_id"], 
    sort_cols=["t_dat"],
    aggs={'article_id': ['list'],
        'price': ['list'],
        'day': ['list'],
        'week': ['list'],
        'month': ['list'],
        'year': ['list'],
        'FN': ['list'],
        'Active': ['list'],
        'club_member_status': ['list'],
        'fashion_news_frequency': ['list'],
        'article_id': ['list'],
        'product_code': ['list'],
        'product_type_no': ['list'],
        'graphical_appearance_no': ['list'],
        'colour_group_code': ['list'],
        'perceived_colour_value_id': ['list'],
        'perceived_colour_master_id': ['list'],
        'department_no': ['list'],
        'index_code': ['list'],
        'index_group_no': ['list'],
        'section_no': ['list'],
        'garment_group_no': ['list']},
    name_sep="_")
list_columns = ['article_id_list',
 'price_list',
 'day_list',
 'week_list',
 'month_list',
 'year_list',
 'FN_list',
 'Active_list',
 'club_member_status_list',
 'fashion_news_frequency_list',
 'product_code_list',
 'product_type_no_list',
 'graphical_appearance_no_list',
 'colour_group_code_list',
 'perceived_colour_value_id_list',
 'perceived_colour_master_id_list',
 'department_no_list',
 'index_code_list',
 'index_group_no_list',
 'section_no_list',
 'garment_group_no_list']
groupby_features = groupby_features[list_columns] >> ListSlice(-SESSIONS_MAX_LENGTH,pad=False) 

stat_features = features >> Groupby(
    groupby_cols=["customer_id"], 
    sort_cols=["t_dat"],
    aggs={'article_id': ['list','count'],
        'price': ['mean', 'std', 'min', 'max']},
    name_sep="_")

groupby_features = (groupby_features + 
                stat_features['article_id_count','price_max','price_min','price_mean','price_std'] + 
                age + 
                customer_id )

filtered = groupby_features >> Filter(f=lambda df: df["article_id_count"] >=SESSIONS_MIN_LENGTH)



# filtered = groupby_features[list_columns] >> Filter(f=lambda df: df['article_id_count'] >= SESSIONS_MIN_LENGTH)
# groupby_features = groupby_features + age + customer_id 


In [12]:
wf = nvt.Workflow(filtered)
train = nvt.Dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'train.parquet'),engine="parquet",  part_size="1GB")
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'valid.parquet'),engine="parquet", part_size="1GB")

In [13]:
train = wf.fit_transform(train)
valid = wf.transform(valid)
train.regenerate_dataset(os.path.join(config.data_final,'transformer4rec', 'train.parquet'),part_size="500MiB", file_size="500MiB")
valid.regenerate_dataset(os.path.join(config.data_final,'transformer4rec', 'valid.parquet'),part_size="500MiB", file_size="500MiB")

3